# Week 5 - Comparing Sequences

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

Minimizers, working with sets, minhash, comparing sequences.

</div>

## Jupyter Shortcuts

You can find a User Guide with a description of features at http://jupyterlab.readthedocs.io

The main interface we'll be using is the Jupyter Notebook interface. If you prefer to run just Jupyter Notebook itself instead of Jupyter Lab, that is fine. 

Some useful hotkeys are:

* Shift-Enter : execute the code in the current cell
* Enter : edit the current cell
* ESC : stop editing a cell and return to "command mode" to use other hotkeys
* m : Turn the current cell into a Markdown cell
* y : Turn the current cell into a code cell
* a : add a new cell above
* b : add a new cell below
* dd : delete the current cell
* c : copy the current cell
* v : paste the copied cell(s)

You can also execute the command `?` or `help()` to get help on any function. For instance, `sorted?` or `help(sorted)`.

## Setup

We will use numpy to create arrays and Biopython to handle DNA sequences.

In [ ]:
import numpy as np
import gzip
from Bio import SeqIO
from Bio.Seq import Seq
import mmh3

In [ ]:
import os
import requests
from IPython.core.display import HTML

# Handy function to fetch our data files
def fetch_file(url, outpath='.'):
    response = requests.get(url)
    if response.status_code == 200:
        print('File found!')
        # Get the filename from the URL
        filename = os.path.basename(url).split('?', 1)[0]
        # Construct the filepath using the specified directory and filename
        filepath = os.path.join(outpath, filename)
        # Create the directory if it doesn't exist
        if not os.path.exists(outpath):
            print(f'Creating output dir: {outpath}')
            os.makedirs(outpath)
        # Check if the file already exists in the specified directory
        if os.path.exists(filepath):
            print(f'{filename} already exists in {outpath}. Skip download.')
        else:
            with open(filepath, 'wb') as f:
                f.write(response.content)
                f.close()
            print(f'Saved to: {filepath}')
    else:
        print(f'File not found: Code {response.status_code}')

In [ ]:
# Load stylesheet
HTML(requests.get('https://raw.githubusercontent.com/melbournebioinformatics/COMP90014/main/data/2023/style/custom.css').text)

## Sequence data 

We'll read in four bacterial genomes to play with.

In [ ]:
# Fetch test data
data = ['NC_000913.fasta.gz','NC_002695.fasta.gz','NC_003197.fasta.gz','NC_021870.fasta.gz']

for filename in data:
    url = f'https://github.com/melbournebioinformatics/COMP90014/blob/main/data/2023/Workshop_05/data/{filename}?raw=true'
    fetch_file(url,outpath='data')


To work with these sequences in Python we will import the contents of the fasta files using the Biopython library:

In [ ]:
# Import with biopython as SeqRecord objects

# List of file paths to your FASTA.gz files
fasta_gz_files = ['data/NC_000913.fasta.gz','data/NC_002695.fasta.gz','data/NC_003197.fasta.gz','data/NC_021870.fasta.gz']

# Create an empty list to store SeqRecord objects
seq_records = []

# Iterate through the list of file paths
for file_path in fasta_gz_files:
    with gzip.open(file_path, "rt") as handle:
        # Use SeqIO to parse the sequences
        for record in SeqIO.parse(handle, "fasta"):
            # Append each SeqRecord to the list
            seq_records.append(record)

# Now you have a list of SeqRecord objects, one for each sequence in all files
# You can access them by index, e.g., seq_records[0] for the first sequence
# Remember we only need the sequences today so will will extract the 'seq' attribute from each SeqRecord.


In [ ]:
# The SeqRecord contains metadata from the fasta file
for record in seq_records:
    print(record + '\n\n')

We only need the sequences today so will will extract the 'seq' attribute from the SeqRecord.

In [ ]:
# View first 100 bases of sequence

for record in seq_records:
    print(record.id)
    print(record.seq[:100] + '\n')

## Working with Sets

In Python sets are an unordered collection of unique elements. Sets are similar to lists and tuples, but unlike lists and tuples, sets cannot contain duplicate values. 

We can use sets for tasks that involve handling unique items, such as removing duplicates from a list or testing membership in a collection.

In [ ]:
# Create sets in Python

empty_set = set()

my_set = {1, 2, 3, 4, 5}

In [ ]:
# Convert a list to a set
set([1,1,2,3,4,4,])

Adding and Removing Elements:

You can add elements to a set using the add() method and remove elements using the remove() or discard() methods.

Note: discard() does not raise an error if if the element is not found

In [ ]:
my_set.add(6)
my_set.remove(3)

print(my_set)

In [ ]:
# Membership Testing: You can test if an element is in a set using the in keyword.

if 2 in my_set:
    print("2 is in the set")

In [ ]:
# You can get the number of elements in a set using the len() function.

len(my_set)

### Set Operations:

Sets support various operations such as union, intersection, difference, and symmetric difference.

Union (|): Combines two sets and returns a new set with all unique elements.

In [ ]:
set1 = {1, 2, 3}
set2 = {3, 4, 5}

union_set = set1 | set2

print(union_set)

Intersection (&): Returns a new set containing elements that are in both sets.

In [ ]:
intersection_set = set1 & set2

print(intersection_set)

Difference (-): Returns a new set with elements from the first set that are not in the second set.

Also called the "exclusion"

In [ ]:
difference_set = set1 - set2

# In set1 and not in set2
print(difference_set)


Symmetric Difference (^): Returns a new set with elements that are in either of the sets but not in both.

In [ ]:
symmetric_difference_set = set1 ^ set2

# Unique to either set, not shared
print(symmetric_difference_set)

# Minimizers

Minimizers are a subset of representative kmers that are selected by some reproducible method - generally be selecting the bottom 's' hash values of all unique kmers in a sequence. 

Minimizers help in reducing the computational complexity of comparing sequences and identifying common elements.


## Exercise 1: Extract minimizers


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> Write a function that returns a Minhash Sketch of a DNA sequence. The sketch should be a set containing the bottom `s` hash values of kmers extracted from the input sequence`.
    
- Input: 
    - A DNA string or Seq object
    - kmer len `k`
    - Number of minimizers to store in the sketch
- Extract canonical kmers from the input seq
- Calculate hash values
- Output: Return set of the 's' smallest hash values
    
</div>

In [ ]:
# Some helper functions for calculating a hash value from a canonical kmer

def get_canon(kmer):
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    rev = kmer[::-1]
    revcomp = ''.join([complement_dict[base] for base in rev])
    if kmer <= revcomp:
        return kmer
    else:
        return revcomp


# Hash canonical kmer 
def dna_hash(kmer):
    ckmer = get_canon(kmer)
    hashval = mmh3.hash64(str(ckmer), seed=42, signed=False)[0]
    return hashval

In [ ]:
# Implement minimiser extraction
dna_hash('TTTA')

In [ ]:
def minhash_sketch(dna_sequence, k, num_minimizers):
    """
    Calulate minhash sketch from DNA sequence.
    """
    ### BEGIN SOLUTION
    minimizers = [float('inf')] * num_minimizers  # Initialize with +∞
    
    for i in range(len(dna_sequence) - k + 1):
        kmer = dna_sequence[i:i + k]
        hash_value = dna_hash(kmer)
        #print(f'{get_canon(kmer)}: {hash_value}')
        
        if hash_value < max(minimizers):
            minimizers.remove(max(minimizers))
            minimizers.append(hash_value)
    
    return sorted(minimizers)

    # Alt solution:
    #hashes = set()
    #
    #for i in range(len(seq) - k + 1):
    #    kmer = seq[i:i+k]
    #    khash = dna_hash(kmer)
    #    # print(f'{get_canon(kmer)}: {hash_value}')  # can uncomment to show kmers & hash values
    #    hashes.add(khash)
    #
    #hash_l = list(hashes)
    #hash_l.sort()
    #sketch = hash_l[:m]

    ### END SOLUTION

In [ ]:
# First 100 bases of our first SeqRecord
minhash_sketch(seq_records[0].seq[:100], 5, 10)


# Estimating distance with Minhash

Genomes that share kmers should share a similar proportion of minimizers
We can rapidly estimate the distance of two sequences by calculating the number of kmers they share
This is called the Minhash distance and is an approimation for the true jaccard distance (if we examined all the kmers)

Remember these distances are probabalistic.

## Exercise 2: True Jaccard Distance

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> Write a function that returns a set of all unique kmers in a sequence. 
    
- Input: 
    - A DNA string or Seq object
    - kmer len `k`
- Extract kmers from the input seq
- Output: Return set of all kmers
    
</div>

In [ ]:
def extract_kmers(seq, k):
    '''
    Extract all kmers. Return Set of kmers.
    '''
    ### BEGIN SOLUTION
    kmers = set()
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        kmers.add(kmer)
    ### END SOLUTION
    
    return kmers


In [ ]:
seq = 'AGTACGGT'
print(extract_kmers(seq, 4)) # should equal {'CGGT', 'ACGG', 'TACG', 'GTAC', 'AGTA'}

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> Calculate jaccard for kmer sets. The Jaccard distance is the `intersection / union`.
    
- Input: Two kmer sets
- Output: Jaccard dist of the sets
    
</div>

In [ ]:
def jaccard(a, b):
    ### BEGIN SOLUTION
    intersection = len(a.intersection(b))
    union = len(a.union(b))
    j = intersection / union
    ### END SOLUTION
    return j


In [ ]:
set1 = set([1, 2, 3, 4, 5])
set2 = set([4, 5, 6, 7, 8])
print(jaccard(set1, set2))    # should equal 0.25

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
    
<b>Challange:</b> Combine the above functions to return jaccard index for two sequences and a kmer size k
    
</div>


In [ ]:
def true_jaccard_distance(seqA, seqB, k):
    ### BEGIN SOLUTION
    kmersA = extract_kmers(seqA, k)
    kmersB = extract_kmers(seqB, k)
    j = jaccard(kmersA, kmersB)
    ### END SOLUTION
    return j


In [ ]:
# identical
seqA = 'AGTACGGTCAGTAGTCGC'
seqB = 'AGTACGGTCAGTAGTCGC'
j = true_jaccard_distance(seqA, seqB, 4)
print(f'identical sequences true jaccard: {j:.2f}')   # should equal 1.0

# single mismatch 
seqA = 'AGTACGGTAAGTAGTCGC'
seqB = 'AGTACGGTTAGTAGTCGC'
j = true_jaccard_distance(seqA, seqB, 4)
print(f'single mismatch true jaccard: {j:.2f}')       # should equal 0.59

# single indel
seqA = 'AGTACGGTAAAGTAGTCGC'
seqB = 'AGTACGGTAAGTAGTCGC'
j = true_jaccard_distance(seqA, seqB, 4) 
print(f'single indel jaccard: {j:.2f}')               # should equal 0.81


## Exercise 3: MinHash Jaccard Distance

Using our minhash_sketch() function from earlier, let's test it on two identical sequences.

In [ ]:
# identical sequences: prove that minhash works & jaccard would be 1
seqA = 'AGTACGGTACATGCGTTGC'
seqB = 'AGTACGGTACATGCGTTGC'

print('\nseqA')
sketchA = minhash_sketch(seqA, 4, 8)
print(f'\nsketchA: {sketchA}')

print('\nseqB')
sketchB = minhash_sketch(seqB, 4, 8)
print(f'\nsketchB: {sketchB}')

In [ ]:
# We can check these are the same using an assert statement
assert sketchA == sketchB

Now lets make the sequences slightly different

In [ ]:
# Different sequences: show that most min 8 hash values match
seqA = 'AGTACGGTACATCCGTTGGGC'
seqB = 'AGTACGGTACATGCGTTGC'

sketchA = minhash_sketch(seqA, 4, 8)
sketchB = minhash_sketch(seqB, 4, 8)

if sketchA != sketchB:
    print('sketches not same!')
    
    
print(f'\nsketchA: {sketchA}')
print(f'\nsketchB: {sketchB}')



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<b>Challange:</b> Calculate the Jaccard distance between two Minhash sketches (Minhash distance)
    
- Input: 
    - Two sets DNA seqs
    - kmer len k
    - Number of minimizers m
- Calculate minhash sketch for each input seq
- Return: Jaccard distance between the two minhash sketches
</div>

In [ ]:
def minhash_jaccard_distance(seqA, seqB, k, m):
    
    ### BEGIN SOLUTION
    sketchA = minhash_sketch(seqA, k, m)
    sketchB = minhash_sketch(seqB, k, m)
    j = jaccard(set(sketchA), set(sketchB))
    ### END SOLUTION
    return j 

In [ ]:
# identical sequences 
print('IDENTICAL')
seqA = 'AGTACGGTACATGCGTTGC'
seqB = 'AGTACGGTACATGCGTTGC'
jt = true_jaccard_distance(seqA, seqB, 4)
jm = minhash_jaccard_distance(seqA, seqB, 4, 8)
print(f'True Jaccard for identical sequences: {jt:.2f}')       # should equal 1.00
print(f'MinHash Jaccard for identical sequences: {jm:.2f}')    # should approximate 1.00
print()

# different sequences: 
# - prove minhash is approximation of jaccard
# - students will all get different values for minhash jaccard based on hash seed
# - compare with each other - jm should all be approximating 0.77
# - run a few times to show different random samples due to hash seed
print('DIFFERENT')
seqA = 'AGTACGGTAGATGCGTTGTGCATGACTGATGCTAGAGTCTGCTACGTAGCGACAGCTTGCAGTCATGC'
seqB = 'AGTACGGTACATGCGTTGTGCACGACTGATGCTAGAGTCTGCTACGTAGCGACAGCTTGCAAGTCATGC'
jt = true_jaccard_distance(seqA, seqB, 4)
jm = minhash_jaccard_distance(seqA, seqB, 4, 8)
print(f'True Jaccard for different sequences: {jt:.2f}')      # should equal 0.77
print(f'MinHash Jaccard for different sequences: {jm:.2f}')   # should approximate 0.77

## Extension challenges

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<b>Challange:</b> Calculate pairwise minhash distances for our 4 bacterial genomes. 

Visualise distances with an MDS plot.
    
Which samples are most similar?
    
Hint: Check the seqrecord descriptions for species names.
</div>

In [ ]:
# Calculate the minhash for each of our bacterial genomes
# If this takes too long to run just consider the first 10,000 bases of the genome
sketch_A = minhash_sketch(seq_records[0].seq[:10000], 5, 100)
sketch_B = minhash_sketch(seq_records[1].seq[:10000], 5, 100)
sketch_C = minhash_sketch(seq_records[2].seq[:10000], 5, 100)
sketch_D = minhash_sketch(seq_records[3].seq[:10000], 5, 100)

In [ ]:
# Get pairwise distances
# Make MDS plot

This workshop was developed by Grace Hall and Adam Taranto.